## TensorFlow API Hierarchy

![TensorFlow API Hierarchy](TF_API_Hierarchy.png)

### Lazy Evaluation

TensorFlow does lazy evaluation: you need to run the graph to get results

`tf.eager` however, allows you to execute operations imperatively

### Graph and Session

Execute TensorFlow graphs by calling run() on a tf.Session

### Evaluating a Tensor

Evaluating a Tensor is a shortcut to calling run() on the graph's default session
`.eval()`

**TensorFlow Eager mode**
![TensorFlow Eager mode](eager_mode.png)

### Visualizing a graph

You can write the graph out using tf.summary.FileWriter

The graph can be visualized in TensorBoard

You can also write to gs:// and start TensorBoard from CloudShell

1. run the following command in Cloud Shell to start TensorBoard:
```
tensorboard --port 8080 --logdir gs://${BUCKET}/${SUMMARY_DIR}
```
2. To open a new browser window, select __Preview on port 8080__ from the __Web preview__ menu in the top-right corner of the Cloud Shell toolbar.
  In the new window, you can use TensorBoard to see the training summary and the visualized network graph.  
  
3. Press Control+C to stop TensorBoard in Cloud Shell
https://cloud.google.com/ml-engine/docs/distributed-tensorflow-mnist-cloud-datalab

## Debugging TensorFlow programs

In [1]:
import tensorflow as tf

In [6]:
def some_method(data):
    a = data[:, 0:2]
    print(a.get_shape())
    #c = data[:, 1]      # addition fail
    c = data[:, 1:3]
    #print(c.get_shape())
    assert len(c.get_shape()) == 2
    s = (a + c)
    return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
    fake_data = tf.constant([
        [5.0, 3.0, 7.1],
        [2.3, 4.1, 4.8],
        [2.8, 4.2, 5.6],
        [2.9, 8.3, 7.3]
    ])
    print(sess.run(some_method(fake_data)))

(4, 2)
[[12.884487 11.878131 12.449096 15.721323]
 [11.878131 10.962208 11.489996 14.509306]
 [12.449096 11.489996 12.043255 15.207892]
 [15.721323 14.509306 15.207892 19.204166]]


### Shape problems

shape problems can often be fixed using

1. tf.reshape()
2. tf.expand_dims()
3. tf.slice()
4. tf.squeeze()

In [9]:
x = tf.constant([[3, 2],
                 [4, 5],
                 [6, 7]])
sliced = tf.slice(x, [0, 1], [2, 1])

with tf.Session() as sess:
    print("sliced: \n", sliced.eval())

sliced: 
 [[2]
 [5]]


In [11]:
t = tf.constant([[[1], [2], [3], [4]], [[5], [6], [7], [8]]])

with tf.Session() as sess:
    print("t")
    print(sess.run(t))
    print(t.get_shape())
    print("t squeezed")
    print(sess.run(tf.squeeze(t)))

t
[[[1]
  [2]
  [3]
  [4]]

 [[5]
  [6]
  [7]
  [8]]]
(2, 4, 1)
t squeezed
[[1 2 3 4]
 [5 6 7 8]]


In [12]:
t = tf.constant([[[1, 2, 3, 4], [5, 6, 7, 8]]])

with tf.Session() as sess:
    print("t")
    print(sess.run(t))
    print(t.get_shape())
    print("t squeezed")
    print(sess.run(tf.squeeze(t)))

t
[[[1 2 3 4]
  [5 6 7 8]]]
(1, 2, 4)
t squeezed
[[1 2 3 4]
 [5 6 7 8]]


### Data type problems

tf.cast()

### Debugging full programs
To debugging full-blown programs, there are three methods:
* tf.Print()
* tfdbg
* TensorBoard

**Change logging level from WARN**
`tf.logging.set_verbosity(tf.logging.INFO)`

**tf.Print() can be used to log specific tensor values**

In [16]:
import tensorflow as tf

def some_method(a, b):
    b = tf.cast(b, tf.float32)
    s = (a / b)  # oops! NaN
    print_ab = tf.Print(s, [a, b])
    s = tf.where(tf.is_nan(s), print_ab, s)
    return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
    fake_a = tf.constant([[5.0, 3.0, 7.1], [2.3, 4.1, 4.8]])
    fake_b = tf.constant([[2, 0, 5], [2, 8, 7]])
    print(sess.run(some_method(fake_a, fake_b)))

[[      nan       nan]
 [      nan 1.4336526]]


In [14]:
! python tfPrint.py

2019-01-30 16:40:59.417061: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
[[5 3 7.1]...][[2 0 5]...]
[[      nan       nan]
 [      nan 1.4336526]]


**use TensorFlow debugger to step through code**

见debug_demo.ipynb
```
import tensorflow as tf
from tensorflow.python import debug as tf_debug

def some_method(a, b):
    b = tf.cast(b, tf.float32)
    s = (a / b)  # oops! NaN
    return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
    fake_a = tf.constant([[5.0, 3.0, 7.1], [2.3, 4.1, 4.8]])
    fake_b = tf.constant([[2, 0, 5], [2, 8, 7]])
    
    sess = tf_debug.LocalCLIDebugWrapperSession(sess)
    sess.add_tensor_filter("has_inf_or_nan", tf_debug.has_inf_or_nan)
    print(sess.run(some_method(fake_a, fake_b)))
```

in a Terminal window
```
python xyz.py --debug
```